##  Optical recognition of handwritten digits dataset

 jest dużym zbiorem ręcznie pisanych cyfr.

## Optical recognition of handwritten digits dataset

 is a large collection of hand-written numbers.

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets


### Wczytanie danych 

Jest wiele sposobów załadowania danych mnist do pamięci. Możemy skorzystać z funkcji `fetch_mldata`. Ważne jest by nie umieszczać danych w katalogu domowym. Możemy użyc katalogu `/tmp`. Ponieważ potrzebujemy danych tyko do odczytu, wystarczy jedna kopia dla wszystkich.

### Loading data

There are many ways to load mnist data into memory. We can use the function `fetch_mldata`. It is important not to put data in the home directory. We can use the directory `/ tmp`. Because we only need the data to be read, one copy is enough for everyone.

In [ ]:
from  sklearn import datasets

In [ ]:
nist = datasets.load_digits()

In [ ]:
plt.imshow(nist.data[13].reshape(8,8))

### Zadanie 1.

Dokonaj podziału danych i etykiet na dwa zbiory - trenujący i testowy o rozmiarach w stosunkiu 7:1.

### Excercise 1.

Divide the data and labels into two sets - training and test with sizes in a ratio of 7: 1.

In [ ]:
### BEGIN SOLUTION

(train, test, train_labels, test_labels) = \
  train_test_split(np.array(nist.data[:,:],dtype=np.float32), \
                   nist.target[:].astype(np.float32), \
                   test_size=1/5., random_state=42)

### END SOLUTION


In [ ]:
assert train.shape == (1437, 64)
assert test.shape ==  (360, 64)
assert train_labels.shape == (1437,)
assert test_labels.shape == (360,)
assert train[123].dtype == np.float32
assert train[123].shape == (64,)

### Zadanie 2.

Wyświetl dla losowego przykładu etykietę oraz cyfrę. Zauważ, że cyfry są obrazkami `8x8`.

### Exercise 2.

Display the label and number for a random example. Note that the numbers are `8x8` pictures.

In [ ]:
### BEGIN SOLUTION
ith = 134
plt.imshow(train[ith].reshape(8,8))
print(train_labels[ith])
### END SOLUTION


In [ ]:
### BEGIN SOLUTION

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(multi_class='ovr')
clf.fit(train,train_labels)
%time print(clf.score(test,test_labels ))

### END SOLUTION


In [ ]:
clf.coef_.shape,clf.intercept_.shape

## NIST z klasyfikatorem kNN

Korzystając z pakietu `sklearn` poklasyfikuj dane MNIST.

*UWAGA* -  testowanie klasyfikatora wymaga kilku sekund na 100 przykładów. Dlatego funkcje: `clf.score` i `clf.predict` należy wywoływać na podziorze danych testujących lub liczyć się z długim czasem działania. 

## NIST with the kNN classification

Use the `sklearn` package to classify MNIST data.

*NOTE* - classifier testing requires a few seconds for 100 examples. Therefore, the functions `clf.score` and` clf.predict` should be called on the test data subdivision or a long operating time should be taken into account.

In [ ]:
### BEGIN SOLUTION

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(7,n_jobs=4)
clf.fit(train,train_labels)
%time print(clf.score(test,test_labels ))

### END SOLUTION


### Predykcja

Znajdź przykłady (np. 10 sztuk) dla których klasyfikator się pomylił. Zastanów się czy człowiek też mógłby się pomylić w tych przypadkach.

### Prediction

Find examples (e.g. 10 items) for which the classifier was wrong. Consider whether man could also be mistaken in these cases.

In [ ]:
np.sum(clf.predict(test) == test_labels),test_labels.shape[0]

In [ ]:
### BEGIN SOLUTION

for ith in range(257):
    predicted = clf.predict( test[ith:ith+1])[0].astype(np.int)
    actual = test_labels[ith].astype(np.int) 
    if predicted != actual:
        print(predicted,actual)
        plt.imshow(test[ith].reshape(8,8))
        plt.show()
### END SOLUTION


##  NIST z klasyfikatorem SVM

Poklasyfikuj dane MNIST klasyfikatorem SVM, korzystając z implementacji `sklearn` 

 - należy zastosować skalowanie danych (np. `preprocessing.StandardScaler`)
 - warto zacząć eksperymenty od `1000` przykładów trenujących. Trenowanie SVM jest wolne.

## NIST with SVM classifier

Classify the MNIST data with an SVM classifier using the implementation of `sklearn`

 - data scaling should be used (e.g. `preprocessing.StandardScaler`)
 - it is worth starting experiments with '1000' training examples. SVM training is slow.

In [ ]:
### BEGIN SOLUTION

from sklearn import svm
from sklearn import preprocessing
scaler = preprocessing.StandardScaler(with_std = True)
scaler.fit(train)
clf = svm.SVC(decision_function_shape='ovo',kernel='rbf')

%time clf.fit(scaler.transform(train),train_labels)

clf.score(scaler.transform(test),test_labels)

### END SOLUTION


In [ ]:
np.sum(clf.predict(scaler.transform(test)) == test_labels),test_labels.shape[0]

##  NIST z klasyfikatorem SVM w zredukowanej przestrzeni danych (PCA)

Wykonaj redukcję danych z pomocą PCA, np.:

    pca = decomposition.PCA(n_components=50, whiten=True,\
                            svd_solver='randomized')
                            
Zmniejszając wymiar przestrzeni można pokusić się o zwiększenie zbioru trenującego!

## NIST with SVM classifier in reduced data space (PCA)

Perform data dimensionality reduction with the help of PCA, e.g .:

    pca = decomposition.PCA (n _components = 50, whiten = True, \
                            svd_ = solver 'randomized')
                            
By reducing the space dimension, you can try to increase the training set!

In [ ]:
### BEGIN SOLUTION

from sklearn import  decomposition

pca = decomposition.PCA(n_components=25, whiten=True,\
                        svd_solver='randomized')
pca.fit(train)
train_pca = pca.transform(train)
test_pca = pca.transform(test)

%time clf.fit(train_pca,train_labels)

%time print(clf.score(test_pca,test_labels))

### END SOLUTION
